In [2]:
# load stock api
# alpaca trade api
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json
import sys
## import json and sys for hide .env file
%matplotlib inline

# Load .env enviroment variables
load_dotenv()

True

In [3]:
# load goog news api
## https://github.com/ranahaani/GNews
## the reference also should consider google's own https://developers.google.com/webmaster-tools/v1/searchanalytics?hl=zh-cn
## to be simpler, use google trends api
## https://www.youtube.com/watch?v=WSnZrEI-sZ0
## https://pypi.org/project/pytrends/

from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)


ModuleNotFoundError: No module named 'pytrends'